In [16]:
import pandas as pd
from os import listdir,mkdir
from os.path import isdir
import matplotlib.pyplot as plt
rcParams = {'font.size': 20, 'font.weight': 'normal', 'font.family': 'sans-serif',
            'axes.unicode_minus':False, 'axes.labelweight':'normal'}

plt.rcParams.update(rcParams)

def Load_PAF(filepath):
    lines = open(filepath).readlines()
    header = ['Query','Qlen','QStart','QEnd','Orientation','Subject','SLen',
              'SStart','SEnd','Matches','AlignLength','MAPQ']
    op = []
    for l in lines:
        l = l.split('\t')[:12]
        op.append(dict(zip(header, l)))
    df = pd.DataFrame(op)
    df[['Qlen','QStart','QEnd','SLen','SStart',
        'SEnd','Matches','AlignLength','MAPQ']] = df[['Qlen','QStart','QEnd','SLen','SStart',
                                                      'SEnd','Matches','AlignLength','MAPQ']].astype('int')
    df['PIdent'] = df['AlignLength']/df['Qlen']*100
    df = df.loc[df.groupby(['Query'])['PIdent'].idxmax()]
    return df


In [2]:
df_containment_clusters = pd.read_csv('/Users/harihara/Downloads/contig_containment_groups_subset.txt', 
                                      sep = '\t')

In [3]:
contigs = df_containment_clusters.groupby('GroupID')['Contig'].apply(list).to_dict()
representatives = df_containment_clusters.groupby('GroupID')['RepresentativeContig'].apply(list).to_dict()

In [5]:
minimap2_path = '/Users/harihara/Mount/reassembly/contig_mapping_paf/'
aligns = {}
for s in listdir(minimap2_path):
    if not s.startswith('Hot'):
        continue
    if 'osa' in s and 'osb' in s:
        continue
    sample_id = s.replace("_contigs_aligned_to","").replace("_osa","").replace("_osb","").replace(".paf","")
    print(sample_id)
    df = Load_PAF(minimap2_path+s)
    df = df[df['PIdent'] >= 60]
    df = df.set_index('Query')
    if 'osa' in s:
        key = 'OSA'
    if 'osb' in s:
        key = 'OSB'
    try:
        aligns[sample_id][key] = df
    except KeyError:
        aligns[sample_id] = {key:df}

    

HotsprSampleMSe4
HotsprSampleOS55
Hotspr20SampleT8
Hotspr20SampleT8
Hotspr20Samplem2
HotsprSampleOSM2
HotsprSampleOSM1
Hotspr2Sampleme2
Hotspr2Sampleme2
HotsprSampleMSe3
Hotspr2SamplePe2
HotsprSampleMSe3
Hotspr20SampleP4
HotsprSampleOSM4
HotsprSampleMS50
Hotspr2Sample149
HotsprottomLayer_2
HotsprSampleMS13
HotsprSampleOS60
HotsprottomLayer_2
HotsprSampleOS55
HotsprSampleMSe2
HotsprSampleMSe1
HotsprSampleOSM4
HotsprSampOS1265
HotsprSampleMS55
HotsprSampleMS65
HotsprSampleMSe2
Hotspr2Sample149
HotsprSampleMS60
HotsprSampOS1265
HotsprSampOS1260
HotsprottomLayer
HotsprSampleMS55
HotsprSampleOSM3
HotsprSampleOS50
Hotspr20SampleT9
HotsprSamplt10cd
HotsprSampleOSM3
Hotspr20SampleP4
HotsprSampleOSM1
HotsprSampleOS65
HotsprOSTMatCore
HotsprSampleOS60
HotsprOSTMatCore
Hotspr2Sample148
Hotspr20SampleT9
HotsprSampleMSe4
Hotspr2SamplePe2
HotsprSampleMS13
HotsprSampleOSM2
Hotspr20Samplet1
HotsprSampleR4cd
HotsprSampleOS65
HotsprottomLayer
HotsprSampleR4cd
Hotspr20Samplet1
HotsprSampleMSe1
Hotspr2Sam

In [6]:
novel_contigs = {}
novel_contig_path = '/Users/harihara/Research-Activities/Data/Hot-Spring/Missing_Contig_Coords/'
samples = listdir(novel_contig_path+'OSA/')
for s in samples:
    print(s)
    df_osa = pd.read_csv(novel_contig_path+'OSA/'+s, sep = "\t")
    del df_osa['Unnamed: 0']
    #df_osa = df_osa.loc[df_osa.groupby(['Contig'])['OSA_PIdent(Parent)'].idxmax()]
    df_osa = df_osa.set_index('Contig')
   
    df_osb = pd.read_csv(novel_contig_path+'OSB/'+s, sep = "\t")
    del df_osb['Unnamed: 0']
    #df_osb = df_osb.loc[df_osb.groupby(['Contig'])['OSB_PIdent(Parent)'].idxmax()]
    df_osb = df_osb.set_index('Contig')
   
    novel_contigs[s.replace(".txt","")] = {'OSA':df_osa,'OSB':df_osb}

HotsprSampleOS60.txt
HotsprSampleMS65.txt
HotsprSampleOSM1.txt
HotsprSampleOSM3.txt
Hotspr2SamplePe2.txt
HotsprSampleOSM2.txt
HotsprSampleOS65.txt
HotsprSampleMS60.txt
Hotspr2Sampleme2.txt
HotsprSampleOSM4.txt
HotsprSampleMS13.txt
HotsprOSTMatCore.txt
HotsprSamplt10cd.txt
HotsprSampleR4cd.txt
Hotspr20Samplet1.txt
Hotspr20Samplem2.txt
HotsprSampleMSe4.txt
HotsprSampOS1260.txt
HotsprSampOS1265.txt
HotsprSampleMSe2.txt
HotsprSampleMSe3.txt
HotsprSampleMSe1.txt
Hotspr20SampleT8.txt
Hotspr20SampleT9.txt
HotsprSampleOS55.txt
HotsprSampleMS50.txt
Hotspr20SampleP4.txt
HotsprottomLayer.txt
Hotspr2Sample148.txt
Hotspr2Sample149.txt
HotsprottomLayer_2.txt
HotsprSampleOS50.txt
HotsprSampleMS55.txt
Hotspr2Sampleee2.txt


In [ ]:
Plots_dir = '/Users/harihara/Mount/Contig_Tiling_3/'
if not isdir(Plots_dir):
    mkdir(Plots_dir)

groups = list(contigs.keys())
for g in groups:
    fig, ax = plt.subplots(2,1,figsize = (20,18))
    containments = list(set(contigs[g] + representatives[g]))
    osa_ctr,osb_ctr,osa_samples,osb_samples = 0,0,[],[]
    missing = 0
    for c in containments:
        splits = c.split('_')
        if len(splits) == 4:
            sample = splits[0]
            contig = splits[2]+'_'+splits[3]
            genome = splits[1].upper()
        elif len(splits) == 5:
            sample = splits[0] +'_'+splits[1]
            contig = splits[3]+'_'+splits[4]
            genome = splits[2].upper()
        try:
            start = list(novel_contigs[sample][genome].loc[contig]['Start'].tolist())
            end = list(novel_contigs[sample][genome].loc[contig]['End'].tolist())
            osa_pident = list(novel_contigs[sample][genome].loc[contig]['OSA_PIdent(Contig)'].tolist())
            osb_pident = list(novel_contigs[sample][genome].loc[contig]['OSB_PIdent(Contig)'].tolist())
            if osa_pident[0] < 35 and osb_pident[0] < 35:
                color = 'C3'
            else:
                color = 'orange'
        except TypeError:
            start = [novel_contigs[sample][genome].loc[contig]['Start']]
            end = [novel_contigs[sample][genome].loc[contig]['End']]
            osa_pident = [novel_contigs[sample][genome].loc[contig]['OSA_PIdent(Contig)']]
            osb_pident = [novel_contigs[sample][genome].loc[contig]['OSB_PIdent(Contig)']]
            if osa_pident[0] < 35 and osb_pident[0] < 35:
                color = 'C3'
            else:
                color = 'orange'
        except KeyError:
            try:
                start = list(aligns[sample][genome].loc[contig]['SStart'].tolist())
                end = list(aligns[sample][genome].loc[contig]['SEnd'].tolist())
                color = 'teal'
            except TypeError:
                start = [aligns[sample][genome].loc[contig]['SStart']]
                end = [aligns[sample][genome].loc[contig]['SEnd']]
                color = 'teal'
            except KeyError:
                missing += 1
                print(c,'missing')
                continue
        if genome == 'OSA':
            osa_samples.append(sample)
            for i in range(len(start)):
                ax[0].plot([start[i],end[i]],[osa_ctr,osa_ctr], marker = 'o', color = color, ms = 15)
            osa_ctr += 1
            
        if genome == 'OSB':
            osb_samples.append(sample)
            for i in range(len(start)):
                ax[1].plot([start[i],end[i]],[osb_ctr,osb_ctr], marker = 'o', color = color, ms = 15)
            osb_ctr += 1
        
    print(g, '\t', len(containments), '\t', missing)
    ax[0].plot(0,0, color = 'teal', label = 'Pident >60% in OSA or OSB')
    ax[0].plot(0,0, color = 'orange', label = 'Pident >35% in OSA or OSB')
    ax[0].plot(0,0, color = 'C3', label = 'Pident <35% in OSA and OSB')
    ax[0].legend()
    
    ax[1].plot(0,0, color = 'teal', label = 'Pident >60% in OSA or OSB')
    ax[1].plot(0,0, color = 'orange', label = 'Pident >35% in OSA or OSB')
    ax[1].plot(0,0, color = 'C3', label = 'Pident <35% in OSA and OSB')
    ax[1].legend()
    
    ax[0].set_yticks(range(len(osa_samples)))
    ax[0].set_yticklabels(osa_samples)
    ax[0].set_xlim(-50000,3000000)
    ax[0].set_title(g + '\n\nOSA-Genome')
    
    ax[1].set_yticks(range(len(osb_samples)))
    ax[1].set_yticklabels(osb_samples)
    ax[1].set_xlim(-50000,3000000)
    ax[1].set_title('OSB-Genome')
    
    fig.tight_layout()
    fig.savefig(Plots_dir+g+'.pdf')
    
    plt.close('all')

HotsprottomLayer_2_osa_k141_6267 missing
Hotspr2Sample149_osb_k141_35203 missing
HotsprSampleOS50_osa_k141_728 missing
Group_1 	 44 	 3
Group_10 	 6 	 0
HotsprSampleMSe1_osb_k141_67509 missing
Hotspr2Sampleee2_osb_k141_33558 missing
HotsprSampleMS60_osb_k141_13618 missing
HotsprSampleMS13_osb_k141_54552 missing
HotsprSampleMSe3_osb_k141_11656 missing
Hotspr2Sampleme2_osb_k141_18993 missing
Group_1000 	 15 	 6
HotsprSampleMSe1_osb_k141_46879 missing
HotsprSampOS1260_osa_k141_6190 missing
HotsprSampleOSM2_osa_k141_4198 missing
HotsprSampleMSe1_osa_k141_6973 missing
HotsprSampleOSM3_osa_k141_16036 missing
HotsprSampleR4cd_osa_k141_27397 missing
HotsprSampleOS50_osb_k141_5085 missing
Group_1001 	 7 	 7
Group_1002 	 7 	 0
Group_1003 	 27 	 0
Group_1004 	 11 	 0
HotsprottomLayer_osa_k141_3592 missing
Group_1005 	 12 	 1
Group_1007 	 14 	 0
Hotspr20Samplem2_osb_k141_14853 missing
Group_1009 	 10 	 1
Hotspr20Samplet1_osa_k141_14767 missing
HotsprSampOS1265_osb_k141_7406 missing
Group_101 	 8 	

Group_1164 	 12 	 0
Group_1165 	 20 	 0
HotsprOSTMatCore_osa_k141_4658 missing
Group_1166 	 10 	 1
Hotspr2Sample148_osa_k141_12078 missing
HotsprSampleOSM1_osb_k141_11474 missing
Hotspr2Sampleme2_osa_k141_19754 missing
HotsprSampleMS65_osa_k141_13565 missing
Hotspr20SampleT8_osa_k141_15431 missing
HotsprSampleR4cd_osa_k141_18157 missing
Group_1169 	 16 	 6
HotsprSampleR4cd_osa_k141_2516 missing
Group_117 	 20 	 1
HotsprSampleMSe1_osa_k141_18728 missing
HotsprSampleOSM4_osa_k141_3294 missing
Group_1170 	 9 	 2
Group_1171 	 10 	 0
Hotspr2SamplePe2_osa_k141_1269 missing
Hotspr2Sample148_osa_k141_11069 missing
HotsprSampOS1260_osb_k141_1993 missing
Group_1172 	 21 	 3
Group_1175 	 20 	 0
Group_1177 	 10 	 0
HotsprSampleMS60_osa_k141_6244 missing
Group_1178 	 13 	 1
Group_1179 	 27 	 0
Group_118 	 8 	 0
Group_1187 	 19 	 0
Hotspr20SampleT8_osb_k141_8061 missing
Group_1189 	 24 	 1
Hotspr20Samplem2_osa_k141_11796 missing
HotsprSampleOSM1_osa_k141_3802 missing
Hotspr20Samplet1_osa_k141_12165 

Group_1326 	 12 	 0
HotsprSampOS1265_osa_k141_8584 missing
HotsprSamplt10cd_osa_k141_4493 missing
HotsprSampOS1265_osb_k141_5439 missing
Group_1327 	 6 	 3
HotsprOSTMatCore_osa_k141_1352 missing
Group_1328 	 7 	 1
Hotspr20Samplet1_osa_k141_13050 missing
HotsprSampOS1265_osa_k141_12046 missing
Group_133 	 7 	 2
HotsprSampleMS60_osb_k141_87 missing
Group_1330 	 13 	 1
Group_1331 	 8 	 0
Group_1333 	 8 	 0
Group_1334 	 15 	 0
Group_1337 	 10 	 0
Group_1338 	 9 	 0
HotsprSampleOSM2_osb_k141_8535 missing
HotsprSampleOS55_osb_k141_2357 missing
HotsprSampOS1260_osb_k141_1101 missing
HotsprSampleMS65_osa_k141_10578 missing
HotsprSampleMS60_osa_k141_2690 missing
Group_134 	 27 	 5
HotsprSampleOSM3_osa_k141_16519 missing
HotsprSampleOS50_osa_k141_5617 missing
Group_1341 	 12 	 2
Group_1344 	 11 	 0
HotsprSampleMS50_osa_k141_19795 missing
HotsprSampleMSe4_osa_k141_19428 missing
Group_1345 	 10 	 2
HotsprottomLayer_osa_k141_4998 missing
HotsprSampleOS65_osa_k141_805 missing
Group_1347 	 20 	 2
Hot

Group_1470 	 8 	 0
Group_1472 	 9 	 0
Group_1473 	 6 	 0
Group_1474 	 6 	 0
Group_1475 	 22 	 0
HotsprottomLayer_osa_k141_3466 missing
HotsprSampleMS60_osa_k141_9521 missing
HotsprottomLayer_2_osa_k141_195 missing
Group_1476 	 19 	 3
HotsprOSTMatCore_osa_k141_1615 missing
Group_1477 	 15 	 1
HotsprottomLayer_2_osa_k141_7270 missing
Group_1478 	 8 	 1
Group_1479 	 10 	 0
Hotspr2Sample148_osb_k141_27308 missing
Group_148 	 23 	 1
Group_1480 	 6 	 0
Group_1482 	 11 	 0
Hotspr2Sample148_osb_k141_10724 missing
Hotspr2SamplePe2_osb_k141_31034 missing
HotsprSampleMSe3_osb_k141_5095 missing
Group_1486 	 14 	 3
Hotspr2SamplePe2_osa_k141_17635 missing
Group_1487 	 6 	 1
HotsprSampleR4cd_osa_k141_3623 missing
Group_1488 	 7 	 1
HotsprSampleMSe1_osa_k141_24417 missing
HotsprSampleOSM2_osa_k141_8041 missing
HotsprSampleOSM4_osa_k141_16872 missing
Hotspr20SampleT9_osa_k141_21633 missing
HotsprSampleMS55_osa_k141_14641 missing
HotsprottomLayer_osa_k141_1526 missing
Group_1490 	 14 	 6
Group_1491 	 10

Group_199 	 8 	 0
Group_2 	 8 	 0
Group_20 	 40 	 0
Hotspr20SampleT9_osa_k141_6824 missing
HotsprSampleOSM4_osb_k141_79840 missing
HotsprSampleMS65_osb_k141_7013 missing
Group_201 	 7 	 3
HotsprSampOS1265_osa_k141_13855 missing
Group_203 	 15 	 1
Group_204 	 14 	 0
Group_207 	 32 	 0
HotsprSamplt10cd_osa_k141_14668 missing
Group_209 	 6 	 1
HotsprSampleOS55_osb_k141_10353 missing
HotsprSampleMS13_osa_k141_24256 missing
HotsprSampleOSM2_osb_k141_5929 missing
HotsprSampOS1265_osb_k141_1146 missing
HotsprSampleOS60_osa_k141_8298 missing
Group_21 	 20 	 5
HotsprSampOS1265_osb_k141_9347 missing
Hotspr20Samplem2_osa_k141_14859 missing
Hotspr20Samplem2_osb_k141_18875 missing
HotsprSampOS1265_osa_k141_2085 missing
Group_210 	 11 	 4
Group_211 	 23 	 0
Hotspr2SamplePe2_osa_k141_15704 missing
HotsprSampleOSM2_osb_k141_10181 missing
Group_214 	 30 	 2
HotsprSampOS1260_osa_k141_18614 missing
HotsprSampleOSM2_osa_k141_15 missing
HotsprSampOS1260_osa_k141_10993 missing
Group_215 	 22 	 3
Group_219 	

HotsprSamplt10cd_osa_k141_2573 missing
Group_378 	 8 	 1
Group_379 	 15 	 0
HotsprSampleOS55_osb_k141_10353 missing
HotsprSampOS1265_osb_k141_1146 missing
HotsprSampleMS13_osa_k141_24256 missing
HotsprSampleOSM2_osb_k141_5929 missing
HotsprSampleOS60_osa_k141_8298 missing
Group_38 	 13 	 5
HotsprSampOS1260_osa_k141_5478 missing
Group_380 	 17 	 1
HotsprOSTMatCore_osb_k141_4485 missing
Group_381 	 14 	 1
HotsprSampleMS60_osb_k141_34084 missing
Hotspr20SampleT9_osb_k141_8874 missing
HotsprSampleOSM3_osa_k141_4329 missing
Hotspr2Sampleme2_osb_k141_6751 missing
Hotspr2Sampleee2_osb_k141_37243 missing
HotsprSampleOS60_osa_k141_7596 missing
HotsprSamplt10cd_osb_k141_72840 missing
HotsprSampleOSM2_osa_k141_4415 missing
HotsprSampOS1260_osa_k141_13690 missing
Hotspr2Sampleee2_osb_k141_44140 missing
HotsprSampleMSe1_osb_k141_50893 missing
HotsprSampleR4cd_osb_k141_34983 missing
Hotspr2Sample148_osb_k141_1967 missing
HotsprSampleMSe3_osb_k141_16045 missing
HotsprSampOS1265_osa_k141_12168 missing

HotsprSampleOS65_osb_k141_42991 missing
Group_510 	 8 	 1
Group_511 	 13 	 0
Group_513 	 15 	 0
Group_515 	 6 	 0
Hotspr2Sample149_osa_k141_12841 missing
Hotspr2Sample148_osa_k141_5076 missing
HotsprSampleMS50_osa_k141_18481 missing
Hotspr2Sampleme2_osa_k141_19839 missing
Group_516 	 28 	 4
HotsprSamplt10cd_osb_k141_19143 missing
HotsprSampleOSM2_osb_k141_15689 missing
HotsprSampOS1260_osb_k141_4283 missing
Group_518 	 18 	 3
HotsprSampOS1265_osa_k141_8634 missing
Group_519 	 17 	 1
HotsprSampleMS60_osa_k141_14322 missing
HotsprSamplt10cd_osa_k141_21544 missing
Group_52 	 7 	 2
HotsprSampOS1265_osb_k141_1551 missing
Group_520 	 12 	 1
HotsprSampleMS13_osa_k141_15483 missing
HotsprSampleMSe2_osa_k141_25873 missing
Hotspr2Sampleee2_osa_k141_31682 missing
Hotspr2Sampleme2_osa_k141_26126 missing
Hotspr20SampleT8_osa_k141_16812 missing
Hotspr20SampleT9_osa_k141_21076 missing
HotsprSampleMS60_osa_k141_16103 missing
HotsprSampleR4cd_osa_k141_8984 missing
Hotspr2SamplePe2_osa_k141_20811 missin

Group_641 	 10 	 0
HotsprSampOS1265_osa_k141_5135 missing
Hotspr20Samplem2_osa_k141_1429 missing
Hotspr2Sample149_osa_k141_4437 missing
HotsprSampleMS55_osa_k141_19104 missing
Hotspr20SampleT8_osa_k141_5145 missing
Group_642 	 35 	 5
HotsprSampleMSe3_osa_k141_13001 missing
Group_644 	 12 	 1
HotsprSampleMS13_osb_k141_65526 missing
HotsprSampleMS55_osa_k141_568 missing
HotsprottomLayer_2_osa_k141_4129 missing
Group_645 	 33 	 3
Group_646 	 21 	 0
Group_649 	 19 	 0
Group_650 	 12 	 0
Hotspr2Sample148_osa_k141_12916 missing
Group_651 	 12 	 1
HotsprSampleMS60_osa_k141_9748 missing
Hotspr2Sample148_osa_k141_3564 missing
Group_653 	 12 	 2
HotsprSampOS1260_osa_k141_21381 missing
Group_654 	 15 	 1
Group_655 	 6 	 0
Group_656 	 15 	 0
HotsprSampOS1260_osa_k141_1206 missing
HotsprSampleMSe2_osb_k141_14125 missing
Hotspr20SampleT8_osa_k141_15609 missing
Hotspr20SampleT8_osa_k141_27947 missing
HotsprSampleOS55_osb_k141_3586 missing
HotsprSampleMSe3_osb_k141_47142 missing
HotsprSampleOS60_osb_k

Group_75 	 7 	 0
HotsprSampleMS60_osb_k141_9188 missing
Hotspr20SampleP4_osb_k141_24415 missing
Hotspr2SamplePe2_osa_k141_15529 missing
Group_750 	 8 	 3
Group_751 	 9 	 0
HotsprSampOS1260_osa_k141_1206 missing
HotsprSampleMSe2_osb_k141_14125 missing
Hotspr20SampleT8_osa_k141_15609 missing
Hotspr20SampleT8_osa_k141_27947 missing
HotsprSampleOS55_osb_k141_3586 missing
HotsprSampleMSe3_osb_k141_47142 missing
HotsprSampleOS60_osb_k141_10729 missing
HotsprSampleMS13_osb_k141_4472 missing
HotsprSampleMSe1_osb_k141_25763 missing
Hotspr2Sampleee2_osa_k141_25354 missing
HotsprSampleOSM2_osa_k141_8742 missing
HotsprSampleMS55_osb_k141_16273 missing
Hotspr20SampleT9_osa_k141_5951 missing
Hotspr20Samplem2_osb_k141_7112 missing
Group_753 	 15 	 14
Hotspr2Sample148_osa_k141_12035 missing
Group_754 	 6 	 1
Hotspr20Samplet1_osa_k141_20355 missing
HotsprSampleMS50_osa_k141_23814 missing
Group_755 	 11 	 2
HotsprSampleMSe3_osa_k141_15762 missing
Group_756 	 35 	 1
Group_758 	 6 	 0
Hotspr20Samplet1_osa